# Summarize Assignment of On-board Survey Demand
This notebook uses the Acceptance Criteria infrastructure to summarize the results of the assignment of the on-board survey demand to the TM2 networks. 

In [17]:
from tm2py.acceptance.canonical import Canonical
from tm2py.acceptance.simulated import Simulated
from tm2py.acceptance.observed import Observed
from tm2py.acceptance.acceptance import Acceptance

import pandas as pd
import geopandas as gpd

### Remote I/O

In [18]:
scenario_config_file = "../examples/scenario_config.toml"
observed_config_file = "../examples/observed_data.toml"
model_config_file = "../examples/model_config.toml"
canonical_crosswalk_file = "../examples/canonical_crosswalk.toml"

input_previous_summary_filename = "on-board-assignment-summary.csv"

output_folder_root = "/Users/wsp/Documents/GitHub/tm2py/examples/temp_on_board_assign/"
output_filename = "on-board-assignment-summary.csv"

## Parameters

In [19]:
append_or_create = "append" 
iteration_description = "Iter 2 with Reduced Boarding Penalties and Smaller Penalties on Local and Express Bus"
iteration_number = 44

In [20]:
c = Canonical(canonical_crosswalk_file, on_board_assign_summary=True)

In [21]:
s = Simulated(canonical=c, scenario_file=scenario_config_file, model_file=model_config_file, on_board_assign_summary=True)
s.reduce_on_board_assignment_boardings(time_period_list=["am"])

In [22]:
s.simulated_boardings_df.head()

,daily_line_name,tm2_mode,line_mode,operator,technology,fare_system,total_boarding,total_hour_cap,time_period,line_name
0,10_573_198,12,b,Emery Go-Round,Local bus,18,29.032267,33.750000,daily,N/A -- Daily Record
1,10_573_199,12,b,Emery Go-Round,Local bus,18,131.823475,207.692308,daily,N/A -- Daily Record
2,10_573_200,12,b,Emery Go-Round,Local bus,18,143.729876,207.692308,daily,N/A -- Daily Record
3,10_573_201,12,b,Emery Go-Round,Local bus,18,25.888117,33.750000,daily,N/A -- Daily Record
4,10_573_202,12,b,Emery Go-Round,Local bus,18,136.368773,192.857143,daily,N/A -- Daily Record


In [23]:
o = Observed(canonical=c, observed_file=observed_config_file, on_board_assign_summary=True)
o.reduce_on_board_survey(read_file_from_disk=True)

In [24]:
a = Acceptance(canonical=c, simulated=s, observed=o, output_file_root=output_folder_root)  
compare_df = a.compare_transit_boardings()

In [25]:
compare_df.head()

,tm2_mode,line_mode,operator,technology,time_period,total_boarding,survey_route,survey_boardings,florida_threshold,daily_line_name,...,line_name,survey_tech,survey_operator,survey_agency,standard_route_id,canonical_operator,standard_route_short_name,standard_route_long_name,standard_line_name,standard_headsign
0,14.0,b,Caltrain,Local Bus,am,46.007467,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14.0,b,Caltrain,Local Bus,daily,46.007467,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,120.0,h,BART,Heavy Rail,am,151552.332603,BART,134161.945802,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,120.0,h,BART,Heavy Rail,daily,151552.332603,BART,431650.949580,0.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,130.0,r,Caltrain,Commuter Rail,am,14881.497511,Caltrain,20978.587956,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Make Output

In [26]:
compare_df["iteration_description"] = iteration_description
compare_df["iteration_number"] = iteration_number

if append_or_create == "create":
    compare_df.to_csv(output_folder_root + output_filename, index=False)
else:
    previous_df = pd.read_csv(output_folder_root + input_previous_summary_filename, dtype=compare_df.dtypes.to_dict())
    pd.concat([previous_df, compare_df], ignore_index=True).to_csv(output_folder_root + output_filename, index=False)